In [1]:
import csv
import json
import emoji
import tweepy
from janome.tokenizer import Tokenizer
from wordcloud import WordCloud
from collections import Counter, defaultdict
from requests_oauthlib import OAuth1Session
from time import sleep
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [2]:
consumer_key = "my consumer_key"
consumer_secret = "my consumer_secret"
access_token = "my access_token"
access_token_secret = "my access_token_secret"

In [3]:
twitter = OAuth1Session(consumer_key, consumer_secret, access_token, access_token_secret)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
timeline_url = "https://api.twitter.com/1.1/statuses/user_timeline.json"
tweet_url = "https://api.twitter.com/1.1/search/tweets.json"

In [5]:
params = {'screen_name': '@g031p068siwate1',
          'exclude_replies': True,
          'include_rts': False,
          'count': 200}

In [6]:
def remove_emoji(src_str):
    return ''.join(c for c in src_str if c not in emoji.UNICODE_EMOJI)

In [7]:
def counter(texts):
    t = Tokenizer("dic.csv",udic_type="simpledic", udic_enc="shift-jis")
    words_count = defaultdict(int)
    words = []
    for text in texts:
        tokens = t.tokenize(text)
        for token in tokens:
            # 品詞から名詞だけ抽出
            pos = token.part_of_speech.split(',')[0]
            if pos in ['名詞']:
                # 必要ない単語を省く(実際の結果を見てから不必要そうな単語を記載しました)
                if token.base_form not in ["こと", "よう", "そう", "これ", "それ"]:
                    words_count[token.base_form] += 1
                    words.append(token.base_form)
    return words_count, words

## 自分のツイートを取得

In [8]:
f_out = open('tweet.txt', 'w')

for j in range(100):
    res = twitter.get(timeline_url, params=params)
    if res.status_code == 200:
        # API残り回数
        limit = res.headers['x-rate-limit-remaining']
        #print("API remain: " + limit)
        if limit == 1:
            sleep(60*15)

        n = 0
        timeline = json.loads(res.text)
        # 各ツイートの本文を表示
        for i in range(len(timeline)):
            if i != len(timeline)-1:# 絵文字があると、wordcloudが使用できないため、削除する
                f_out.write(remove_emoji(timeline[i]['text']) + '\n')
            else:# 絵文字があると、wordcloudが使用できないため、削除する
                f_out.write(remove_emoji(timeline[i]['text']) + '\n')
                # 一番最後のツイートIDをパラメータmax_idに追加
                params['max_id'] = timeline[i]['id']-1
f_out.close()

## 入力されたキーワードのツイートを取得

In [9]:
f_out = open('keyword.txt', 'w')

print("何を調べますか?")
keyword = input('>> ')
print('----------------------------------------------------')

params = {'q' : keyword, 'count' : 50}
req = twitter.get(tweet_url, params = params)
if req.status_code == 200:
    public_tweets = api.home_timeline()
    for tweet in public_tweets:
            f_out.write(remove_emoji(tweet.text).replace('\ufe0f','').replace('\xa5','') + '\n')
else:
    print("ERROR: %d" % req.status_code)

f_out.close()

何を調べますか?


>>  マクドナルド


----------------------------------------------------


## wordcloud

In [10]:
with open('tweet.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    #DancingMen-SherlockHolmes-Japanese.txt
    texts = []
    for row in reader:
        if(len(row) > 0):
            text = row[0].split('http')
            texts.append(text[0])

words_count, words = counter(texts)
text = ' '.join(words)

In [11]:
# fontは自分の端末にあるものを使用する
fpath = r"C:\home\Fonts\RictyDiminished-for-Powerline-master\RictyDiminished-Bold.ttf"
wordcloud = WordCloud(background_color="white",font_path=fpath, width=900, height=500).generate(text)

In [12]:
wordcloud.to_file("./wordcloud_tweet.png")

In [13]:
with open('keyword.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    #DancingMen-SherlockHolmes-Japanese.txt
    texts = []
    for row in reader:
        if(len(row) > 0):
            text = row[0].split('http')
            texts.append(text[0])

words_count, words = counter(texts)
text = ' '.join(words)

In [14]:
# fontは自分の端末にあるものを使用する
fpath = r"C:\home\Fonts\RictyDiminished-for-Powerline-master\RictyDiminished-Bold.ttf"
wordcloud = WordCloud(background_color="white",font_path=fpath, width=900, height=500).generate(text)

In [15]:
wordcloud.to_file("./wordcloud_keyword.png")

In [16]:
with open('DancingMen-SherlockHolmes-Japanese.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    #DancingMen-SherlockHolmes-Japanese.txt
    texts = []
    for row in reader:
        if(len(row) > 0):
            text = row[0].split('http')
            texts.append(text[0])

words_count, words = counter(texts)
text = ' '.join(words)

In [17]:
# fontは自分の端末にあるものを使用する
fpath = r"C:\home\Fonts\RictyDiminished-for-Powerline-master\RictyDiminished-Bold.ttf"
wordcloud = WordCloud(background_color="white",font_path=fpath, width=900, height=500).generate(text)

In [18]:
wordcloud.to_file("./wordcloud_SherlockHolmes.png")